In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/CSV/cybersecurity_attacks.csv")
data.head(3)

,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,...,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
0,5/30/2023 6:33,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,...,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,NaN,Server
1,8/26/2020 7:08,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,...,Blocked,Low,Sumer Rana,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Bilaspur, Nagaland",NaN,Log Data,NaN,Firewall
2,11/13/2022 8:23,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,...,Ignored,Low,Himmat Karpe,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Segment C,"Bokaro, Rajasthan",114.133.48.179,Log Data,Alert Data,Firewall


In [ ]:
from sklearn.preprocessing import LabelEncoder

# List of text-heavy columns to drop
columns_to_drop = ['Payload Data', 'Device Information', 'Geo-location Data']

# Encode remaining categorical columns
columns_to_encode = [
    'Traffic Type', 'Malware Indicators', 'Alerts/Warnings',
    'Attack Signature', 'Severity Level', 'Network Segment',
    'Firewall Logs', 'IDS/IPS Alerts', 'Log Source'
]

label_encoders = {}
for column in columns_to_encode:
    if column in data.columns:
        le = LabelEncoder()
        # Fill NaN with a placeholder for encoding
        data[column] = data[column].fillna('Unknown')
        data[column] = le.fit_transform(data[column].astype(str))
        label_encoders[column] = le

# Drop unnecessary columns
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')


# Ensure all columns are numeric
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns
if len(non_numeric_columns) > 0:
    print("Non-numeric columns and their unique values:", {col: data[col].unique() for col in non_numeric_columns})
    #raise ValueError("Some columns are still non-numeric after processing!")

print("All columns are now numeric!")


categorical_columns = ['Protocol', 'Packet Type', 'Attack Type', 'Action Taken']

# Encode categorical columns using Label Encoding
label_encoders = {}
for column in categorical_columns:
    if column in data.columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))  # Convert to string if necessary
        label_encoders[column] = le

# Handle IP addresses separately (if present)
ip_columns = ['Source IP Address', 'Destination IP Address']
for column in ip_columns:
    if column in data.columns:
        le = LabelEncoder()
        data[column] = le.fit_transform(data[column].astype(str))  # Convert IP addresses to numbers
        label_encoders[column] = le

# Drop other irrelevant columns (e.g., 'Timestamp', 'User Information', etc.)
columns_to_drop = ['Timestamp', 'User Information', 'Proxy Information']
data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], errors='ignore')

# Ensure all columns are numeric
non_numeric_columns = data.select_dtypes(exclude=[np.number]).columns
if len(non_numeric_columns) > 0:
    print("Non-numeric columns detected:", non_numeric_columns)
    raise ValueError("Some columns are still non-numeric after processing!")

Non-numeric columns and their unique values: {'Timestamp': array(['5/30/2023 6:33', '8/26/2020 7:08', '11/13/2022 8:23', ...,
       '3/31/2022 1:45', '9/22/2023 18:32', '10/10/2023 11:59'],
      dtype=object), 'Source IP Address': array(['103.216.15.12', '78.199.217.198', '63.79.210.48', ...,
       '162.35.217.57', '208.72.233.205', '14.102.21.108'], dtype=object), 'Destination IP Address': array(['84.9.164.252', '66.191.137.154', '198.219.82.17', ...,
       '98.107.0.15', '173.79.112.252', '109.198.45.7'], dtype=object), 'Protocol': array(['ICMP', 'UDP', 'TCP'], dtype=object), 'Packet Type': array(['Data', 'Control'], dtype=object), 'Attack Type': array(['Malware', 'DDoS', 'Intrusion'], dtype=object), 'Action Taken': array(['Logged', 'Blocked', 'Ignored'], dtype=object), 'User Information': array(['Reyansh Dugal', 'Sumer Rana', 'Himmat Karpe', ..., 'Adira Madan',
       'Rati Dara', 'Zaina Kumar'], dtype=object), 'Proxy Information': array(['150.9.97.135', nan, '114.133.48.179', .

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Load and preprocess the data
data = pd.read_csv("/content/drive/MyDrive/CSV/cybersecurity_attacks.csv")

# (Include your preprocessing steps here as shown in your previous code)

# Step 1: Separate features and target
X = data.drop(columns=['Malware Indicators'])
y = data['Malware Indicators']


#Drop the 'Timestamp' column from your features
X = X.drop(columns=['Timestamp'], errors='ignore')



# Ensure all columns are numeric
for column in X.columns:
    if X[column].dtype == 'object':  # Check if column is of type object (string)
        try:
            X[column] = pd.to_numeric(X[column])  # Attempt to convert to numeric
        except ValueError:
            print(f"Column '{column}' contains non-numeric values. Applying Label Encoding.")
            le = LabelEncoder()
            X[column] = le.fit_transform(X[column].astype(str))





# Assume X_flat was intended to be X
# If X_flat was supposed to be a transformed version of X, apply the necessary transformations here
X_flat = X

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_flat, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_rf, y_train_rf)

# Predict on test data
y_pred_rf = rf_model.predict(X_test_rf)

# Evaluate the model
accuracy_rf = accuracy_score(y_test_rf, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)
print(classification_report(y_test_rf, y_pred_rf))

Column 'Source IP Address' contains non-numeric values. Applying Label Encoding.
Column 'Destination IP Address' contains non-numeric values. Applying Label Encoding.
Column 'Protocol' contains non-numeric values. Applying Label Encoding.
Column 'Packet Type' contains non-numeric values. Applying Label Encoding.
Column 'Traffic Type' contains non-numeric values. Applying Label Encoding.
Column 'Payload Data' contains non-numeric values. Applying Label Encoding.
Column 'Alerts/Warnings' contains non-numeric values. Applying Label Encoding.
Column 'Attack Type' contains non-numeric values. Applying Label Encoding.
Column 'Attack Signature' contains non-numeric values. Applying Label Encoding.
Column 'Action Taken' contains non-numeric values. Applying Label Encoding.
Column 'Severity Level' contains non-numeric values. Applying Label Encoding.
Column 'User Information' contains non-numeric values. Applying Label Encoding.
Column 'Device Information' contains non-numeric values. Applying 

ValueError: Input contains NaN